In [1]:
import pandas as pd
import numpy as np

### Load Data

In [2]:
confirmed_xlsx = '../data/confirmed.xlsx'
recovered_xlsx = '../data/recovered.xlsx'
deaths_xlsx = '../data/deaths.xlsx'

In [3]:
confirmed = pd.read_excel(confirmed_xlsx)
recovered = pd.read_excel(recovered_xlsx)
deaths = pd.read_excel(deaths_xlsx)

In [4]:
confirmed.head()

,Province/State,Country/Region,First Confirmed Date,Lat,Long,2020-01-21 22:00:00,2020-01-22 12:00:00,2020-01-23 12:00:00,2020-01-24 12:00:00,2020-01-25 22:00:00,2020-01-26 23:00:00,2020-01-27 20:30:00,2020-01-28 23:00:00,2020-01-29 21:00:00,2020-01-30 11:00:00,2020-01-31 19:00:00,2020-02-01 10:00:00,2020-02-02 21:00:00,2020-02-03 21:00:00,2020-02-04 22:00:00
0,Anhui,Mainland China,2020-01-03,31.82571,117.2264,0,1,9,15,60,70,106,152,200,200,237,297,408,480,530
1,Beijing,Mainland China,2020-01-03,40.18238,116.4142,10,14,22,36,51,68,80,91,111,114,139,168,191,212,228
2,Chongqing,Mainland China,2020-01-03,30.05718,107.8740,5,6,9,27,75,110,132,147,165,182,238,247,300,337,366
3,Fujian,Mainland China,2020-01-03,26.07783,117.9895,0,1,5,10,18,35,59,82,101,101,120,144,159,179,194
4,Gansu,Mainland China,2020-01-03,36.06110,103.8343,0,0,2,2,7,14,19,24,26,26,35,35,51,55,57


### Melt each dataframe so date columns and counts are rows

In [5]:
### 

confirmed = confirmed.melt(id_vars=["Province/State", "Country/Region", "First Confirmed Date", "Lat", "Long"], 
        var_name="Date", 
        value_name="Confirmed Cases").sort_values(by=['Province/State', 'Date'])
recovered = recovered.melt(id_vars=["Province/State", "Country/Region", "First Confirmed Date", "Lat", "Long"], 
        var_name="Date", 
        value_name="Recovered Cases").sort_values(by=['Province/State', 'Date'])
deaths = deaths.melt(id_vars=["Province/State", "Country/Region", "First Confirmed Date", "Lat", "Long"], 
        var_name="Date", 
        value_name="Death Cases").sort_values(by=['Province/State', 'Date'])

confirmed.head()

,Province/State,Country/Region,First Confirmed Date,Lat,Long,Date,Confirmed Cases
0,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-21 22:00:00,0
65,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-22 12:00:00,1
130,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-23 12:00:00,9
195,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-24 12:00:00,15
260,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-25 22:00:00,60


### Check to make sure all data frames have the same number of rows

In [6]:
confirmed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 940
Data columns (total 7 columns):
Province/State          975 non-null object
Country/Region          975 non-null object
First Confirmed Date    975 non-null datetime64[ns]
Lat                     975 non-null float64
Long                    975 non-null float64
Date                    975 non-null datetime64[ns]
Confirmed Cases         975 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 60.9+ KB


In [7]:
recovered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 940
Data columns (total 7 columns):
Province/State          975 non-null object
Country/Region          975 non-null object
First Confirmed Date    975 non-null datetime64[ns]
Lat                     975 non-null float64
Long                    975 non-null float64
Date                    975 non-null datetime64[ns]
Recovered Cases         975 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 60.9+ KB


In [8]:
deaths.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 940
Data columns (total 7 columns):
Province/State          975 non-null object
Country/Region          975 non-null object
First Confirmed Date    975 non-null datetime64[ns]
Lat                     975 non-null float64
Long                    975 non-null float64
Date                    975 non-null datetime64[ns]
Death Cases             975 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 60.9+ KB


### Merge all data frames to one data frame

In [9]:
df = pd.concat([confirmed, recovered, deaths], axis=1)
### Remove duplicated columns
df = df.iloc[:, [0,1,2,3,4,5,6,13,20]]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 9 columns):
Province/State          975 non-null object
Country/Region          975 non-null object
First Confirmed Date    975 non-null datetime64[ns]
Lat                     975 non-null float64
Long                    975 non-null float64
Date                    975 non-null datetime64[ns]
Confirmed Cases         975 non-null int64
Recovered Cases         975 non-null int64
Death Cases             975 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(3), object(2)
memory usage: 76.2+ KB


### Sort data by Province/State then by Date

In [11]:
df = df.sort_values(by=['Province/State', 'Date'])

In [12]:
df.head()

,Province/State,Country/Region,First Confirmed Date,Lat,Long,Date,Confirmed Cases,Recovered Cases,Death Cases
0,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-21 22:00:00,0,0,0
65,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-22 12:00:00,1,0,0
130,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-23 12:00:00,9,0,0
195,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-24 12:00:00,15,0,0
260,Anhui,Mainland China,2020-01-03,31.82571,117.2264,2020-01-25 22:00:00,60,0,0


### Set up MongoDB Connection

In [49]:
import pymongo
import dns
username = 'Terra925'
password = 'H%40mmond271'

In [50]:
conn = 'mongodb+srv://' + username +':' + password + '@cluster0-paegd.mongodb.net/test?retryWrites=true&w=majority'
client = pymongo.MongoClient(conn)
db = client['corona_virus']
collection = db['cases']

### Itterate through rows of df, create a post object, then post new document to MongoDB database

In [20]:

for index, row in df.iterrows():
    post = {
        'location': row['Province/State'],
        'region': row['Country/Region'],
        'firstConfirmedDate': row['First Confirmed Date'],
        'lat': row['Lat'],
        'lng': row['Long'],
        'cases': {
            'date': row['Date'],
            'confirmed': row['Confirmed Cases'],
            'recovered': row['Recovered Cases'],
            'deaths': row['Death Cases']
        }
    }
    id = collection.insert_one(post).inserted_id


In [17]:
post

{'location': 'Zhejiang',
 'region': 'Mainland China',
 'firstConfirmedDate': Timestamp('2020-01-03 00:00:00'),
 'lat': 29.18251,
 'lng': 120.0985,
 'cases': {'date': Timestamp('2020-02-04 22:00:00'),
  'confirmed': 895,
  'recovered': 63,
  'deaths': 0}}

### Create a new collection with dates as the indexes

In [53]:
collection = db['cases_by_date']

#### Create DF that groups by dates then by province/state

In [25]:
by_date = df.groupby(["Date", "Province/State"]).max().reset_index()
by_date.head()



,Date,Province/State,Country/Region,First Confirmed Date,Lat,Long,Confirmed Cases,Recovered Cases,Death Cases
0,2020-01-21 22:00:00,Anhui,Mainland China,2020-01-03,31.82571,117.2264,0,0,0
1,2020-01-21 22:00:00,Arizona,US,2020-01-26,34.04890,-111.0940,0,0,0
2,2020-01-21 22:00:00,Beijing,Mainland China,2020-01-03,40.18238,116.4142,10,0,0
3,2020-01-21 22:00:00,"Boston, MA",US,2020-02-02,42.36010,-71.0589,0,0,0
4,2020-01-21 22:00:00,British Columbia,Canada,2020-01-28,49.28270,-123.1210,0,0,0


#### Create DF that groups by dates

In [24]:
totals = by_date.groupby("Date")["Confirmed Cases", "Recovered Cases", "Death Cases"].max().reset_index()
totals.head()

,Date,Confirmed Cases,Recovered Cases,Death Cases
0,2020-01-21 22:00:00,270,25,0
1,2020-01-22 12:00:00,444,28,0
2,2020-01-23 12:00:00,444,28,0
3,2020-01-24 12:00:00,549,31,24
4,2020-01-25 22:00:00,1052,42,52


### Create the documents and upload to MongoDB Collection

In [55]:
### iterate through totals df so there is only one document per day
for index, row in totals.iterrows():
    date = row['Date']
    ### base post object
    post = {
        'date': row['Date'],
        'total_confirmed': row['Confirmed Cases'],
        'total_recovered': row['Recovered Cases'],
        'total_deaths': row['Death Cases'],
        'locations': {} ### this will be updated in the next itteration
        }
    ### iterate through by_date df to store relevant data
    for i, r in by_date.iterrows():
        ### only add to post if the date in by_date df matches the date in current itteration of totals df
        if r['Date'] == date:
            location = r['Province/State']
            ### temporary object to hold data that will be added to "locations" in post
            obj = {
                        "region": r['Country/Region'],
                        "lat": r["Lat"],
                        "lng": r["Long"],
                        "confirmed": r["Confirmed Cases"],
                        "recovered": r["Recovered Cases"],
                        "deaths": r["Death Cases"]
                    }
            ### add obj data to posted data
            post['locations'][location] = obj
            
            ### upload post to Mongo DB
        else:
            
            continue
    id = collection.insert_one(post).inserted_id
